# Assembly: mapping assembly code to C (continued); tracing assembly code
_COSC 208, Introduction to Computer Systems, 2024-03-25_

## Announcements
* Project 2 due Wednesday @ 11pm
    * **Functionality**
        * Satisfactory: **two or fewer minor bugs** (see examples in project instructions)
        * Progressing: **two or fewer major bugs**
    * **Design**
        * Satisfactory: must adhere to **all** design criteria in the project instructions
        * Progressing: adheres to **most, but not all,** of the design criteria
    * **Academic Integrity**
        * “Set[ting] an example of ethical behavior in public and in private” is one of the 13 goals of a Colgate education. As part of this goal, Colgate has an **academic honor code** you are expected to follow. Additionally, “a computing professional should uphold, promote, and respect the principles of the **Association for Computing Machinery (ACM) Code of Ethics and Professional Conduct”**.
        * You **may not share or copy code/answers from other students.** Violations of this policy will be considered facilitating academic misconduct and/or cheating according to the academic honor code.
        * You **must clearly indicate if any code is copied from or inspired by external sources** (e.g., Stack Overflow, ChatGPT, GitHub Copilot) and adhere to the licensing requirements of such code. Violations of this policy will be considered cheating or plagiarism according to Colgate’s academic honor code.
        * **If you are struggling—due to a lack of understanding, time pressure, personal matters, etc.—please contact me. I want you to succeed, and I will do everything I can to help you.**

## Outline
* Warm-up
* Relationship between C variables and assembly code
* Instruction semantics
* Tracing assembly code

## Warm-up

* Q1: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  long deref(long *p) {
/* 3*/      long v = *p;
/* 4*/      return v;
/* 5*/  }
/* 6*/  int main() {
/* 7*/      long x = 2;
/* 8*/      long y = deref(&x);
/* 9*/      printf("y = %ld\n", y);
/*10*/      return 0;
/*11*/  }

y = 2


```
0000000000000854 <deref>:
 854:   d10083ff    sub sp, sp, #0x20 // 2: Create stack frame 
 858:   f90007e0    str x0, [sp, #8]  // 2: Store parameter
 85c:   f94007e0    ldr x0, [sp, #8]  // 3: Load pointer
 860:   f9400000    ldr x0, [x0]      // 3: Dereference pointer
 864:   f9000fe0    str x0, [sp, #24] // 3: Store variable
 868:   f9400fe0    ldr x0, [sp, #24] // 4: Load return value
 86c:   910083ff    add sp, sp, #0x20 // 4: Destroy stack frame
 870:   d65f03c0    ret               // 4: Return to caller
```

```
0000000000000854 <deref>:
 854:   d10083ff    sub sp, sp, #0x20
 858:   f90007e0    str x0, [sp, #8]
 85c:   f94007e0    ldr x0, [sp, #8]
 860:   f9400000    ldr x0, [x0]
 864:   f9000fe0    str x0, [sp, #24]
 868:   f9400fe0    ldr x0, [sp, #24]
 86c:   910083ff    add sp, sp, #0x20
 870:   d65f03c0    ret
```

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Relationship between C variables and assembly code

The following C program (`operands.c`) has been compiled into assembly:

In [ ]:
int operandsA(int a) {
    return a;
}
long operandsB(long b) {
    return b;
}
int operandsC(int *c) {
    return *c;
}
long operandsD(long *d) {
    return *d;
}
int main() {
    operandsA(5);
    operandsB(5);
    int x = 5;
    operandsC(&x);
    long y = 5;
    operandsD(&y);
}

```
00000000000007ec <operandsA>:
    7ec:    d10043ff     sub    sp, sp, #0x10
    7f0:    b9000fe0     str    w0, [sp, #12]
    7f4:    b9400fe0     ldr    w0, [sp, #12]
    7f8:    910043ff     add    sp, sp, #0x10
    7fc:    d65f03c0     ret    
0000000000000800 <operandsB>:
    800:    d10043ff     sub    sp, sp, #0x10
    804:    f90007e0     str    x0, [sp, #8]
    808:    f94007e0     ldr    x0, [sp, #8]
    80c:    910043ff     add    sp, sp, #0x10
    810:    d65f03c0     ret
0000000000000814 <operandsC>:
    814:    d10043ff     sub    sp, sp, #0x10
    818:    f90007e0     str    x0, [sp, #8]
    81c:    f94007e0     ldr    x0, [sp, #8]
    820:    b9400000     ldr    w0, [x0]
    824:    910043ff     add    sp, sp, #0x10
    828:    d65f03c0     ret
000000000000082c <operandsD>:
    82c:    d10043ff     sub    sp, sp, #0x10
    830:    f90007e0     str    x0, [sp, #8]
    834:    f94007e0     ldr    x0, [sp, #8]
    838:    f9400000     ldr    x0, [x0]
    83c:    910043ff     add    sp, sp, #0x10
    840:    d65f03c0     ret
```

<div style="page-break-after:always;"></div>

* Q2: _How does the assembly code for `operandsA` and `operandsB` differ? Why?_

* `operandsA` takes and returns an int, which is 32-bits, whereas `operandsB` takes and returns a long, which is 64-bits, so:
    * `operandsA` uses `w0` while `operandsB` uses `x0`
    * `operandsA` stores the parameter at `sp + 12` while `operandsB` stores the parameter at `sp + 8`

<p style="height:6em;"></p>

* Q3: _How does the assembly code for `operandsB` and `operandsD` differ? Why?_

* `operandsB` takes and returns a long, whereas `operandsD` takes a pointer to a long and returns a long, so:
    * `operandsD` must deference the pointer (`ldr x0, [x0]`) before returning 

<p style="height:6em;"></p>

* Q4: _How does the assembly code for `operandsC` and `operandsD` differ? Why?_

* `operandsC` takes a pointer to an int and returns and int, whereas `operandsD` takes a pointer to a long and returns a long
* both take a memory address (a 64-bit value), which is initially in `x0` and stored at `sp + 8`
* the dereference of the pointer is a 32-bit value in `operandsC` and a 64-bit value in `operandsD`, so the value is loaded into `w0` in `operandsC` and `x0` in `operandsD`

<p style="height:6em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

## Instruction semantics

_Use the provided ARM64 reference sheet to answer the following questions._
* Q5: _What is the difference between a `neg` instruction and a `mvn` instruction?_

    neg negates a number, whereas mvn performs a bitwise not

<div style="height:6em;"></div>

* Q6: _What is the difference between an `orr` instruction and a `eor` instruction?_

    orr performs a bitwise or, whereas eor performs a bitwise xor

<div style="height:6em;"></div>

* Q7: _What is the difference between a `lsr` instruction and a `ldr` instruction?_

    lsr performs a right shift, whereas ldr copies a value from memory into a register

<div style="height:6em;"></div>

* Q8: _What is the difference between a `add` instruction and a `and` instruction?_

    add performs addition, whereas and performs bitwise and

<div style="height:6em;"></div>

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Tracing assembly code example

* Example

```
0000000000000854 <deref>:
 854:   d10083ff    sub sp, sp, #0x20
 858:   f90007e0    str x0, [sp, #8]
 85c:   f94007e0    ldr x0, [sp, #8]
 860:   f9400000    ldr x0, [x0]
 864:   f9000fe0    str x0, [sp, #24]
 868:   f9400fe0    ldr x0, [sp, #24]
 86c:   910083ff    add sp, sp, #0x20
 870:   d65f03c0    ret
```

* Contents of the stack and registers when the `deref` function (from the warm-up) starts executing
    ```
    pc = 0x854
    sp = 0xFB0
    w/x0 = 0xFB8
    ```
    ```
     sp -> +--+--+--+--+--+--+--+--+
     0xFB0 |                       |
           +--+--+--+--+--+--+--+--+
     0xFB8 |           2           |
           +--+--+--+--+--+--+--+--+
     0xFC0 |         0x770         |
           +--+--+--+--+--+--+--+--+
     0xFC8 |                       |
           +--+--+--+--+--+--+--+--+
    ```

* Contents of the stack and registers *before* executing the second-to-last assembly instruction (`add sp, sp, #0x20`)
    ```
    pc = 0x86c
    sp = 0xF90
    w/x0 = 2
    ```
    ```
           0  1  2  3  4  5  6  7  8
     sp -> +--+--+--+--+--+--+--+--+
     0xF90 |                       |
           +--+--+--+--+--+--+--+--+
     0xF98 |         0xFB8         |
           +--+--+--+--+--+--+--+--+
     0xFA0 |           2           |
           +--+--+--+--+--+--+--+--+
     0xFA8 |                       |
           +--+--+--+--+--+--+--+--+
     0xFB0 |                       |
           +--+--+--+--+--+--+--+--+
     0xFB8 |           2           |
           +--+--+--+--+--+--+--+--+
     0xFC0 |         0x770         |
           +--+--+--+--+--+--+--+--+
     0xFC8 |                       |
           +--+--+--+--+--+--+--+--+
    ```

## Extra practice

* Q9: _For each line of assembly, indicate which original line of C code the assembly instruction was derived from._

In [ ]:
/*1*/  int sum(int a, int *b) {
/*2*/     int c = *b;
/*3*/     int d = a + c;
/*4*/     return d;
/*5*/  }

```
0000000000000584 <sum>:
  584:    d10083ff     sub sp, sp, #0x20   // 1
  588:    b9001fe0     str w0, [sp, #28]   // 1
  58c:    f9000be1     str x1, [sp, #16]   // 1
  590:    f9400be8     ldr x8, [sp, #16]   // 2
  594:    b9400109     ldr w9, [x8]        // 2
  598:    b9000fe9     str w9, [sp, #12]   // 2
  59c:    b9401fe9     ldr w9, [sp, #28]   // 3
  5a0:    b9400fea     ldr w10, [sp, #12]  // 3
  5a4:    0b0a0129     add w9, w9, w10     // 3
  5a8:    b9000be9     str w9, [sp, #8]    // 3
  5ac:    b9400be0     ldr w0, [sp, #8]    // 4
  5b0:    910083ff     add sp, sp, #0x20   // 4
  5b4:    d65f03c0     ret                 // 4
```

```
0000000000000584 <sum>:
  584:    d10083ff     sub sp, sp, #0x20
  588:    b9001fe0     str w0, [sp, #28]
  58c:    f9000be1     str x1, [sp, #16]
  590:    f9400be8     ldr x8, [sp, #16]
  594:    b9400109     ldr w9, [x8]
  598:    b9000fe9     str w9, [sp, #12]
  59c:    b9401fe9     ldr w9, [sp, #28]
  5a0:    b9400fea     ldr w10, [sp, #12]
  5a4:    0b0a0129     add w9, w9, w10
  5a8:    b9000be9     str w9, [sp, #8]
  5ac:    b9400be0     ldr w0, [sp, #8]
  5b0:    910083ff     add sp, sp, #0x20
  5b4:    d65f03c0     ret
```